In [25]:
import cv2
import numpy as np


In [26]:
img1 = cv2.imread('data/IMG_8297.jpg')
img2 = cv2.imread('data/IMG_8298.jpg')
img1 = cv2.resize(img1, ((int)(img1.shape[1]/4), (int)(img1.shape[0]/4)))
h,w, c = img1.shape

# resize img2 to match img1 
img2= cv2.resize(img2, (w, h), interpolation=cv2.INTER_LINEAR)

img12 = np.hstack((img1, img2))

cv2.namedWindow('OriginalImage', cv2.WINDOW_NORMAL)
cv2.imshow('OriginalImage', img12)
cv2.waitKey(0)

-1

In [27]:
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

img1_kp = cv2.drawKeypoints(img1, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img2_kp = cv2.drawKeypoints(img2, kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

img12_kp = np.hstack((img1_kp, img2_kp))
cv2.namedWindow('FeaturePoints', cv2.WINDOW_NORMAL)
cv2.imshow('FeaturePoints', img12_kp)
cv2.waitKey(0)

-1

In [28]:
bf = cv2.BFMatcher()
matches = bf.match(des1, des2)
matches = sorted(matches, key = lambda x:x.distance)

img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:200], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.namedWindow('MatchedPoints', cv2.WINDOW_NORMAL)
cv2.imshow('MatchedPoints', img_matches)
cv2.waitKey(0)

-1

### 最近隣距離比により、良いマッチングペアを選出する

In [29]:
matches_rt = bf.knnMatch(des1, des2, k=2)
good_matches = []
for m,n in matches_rt:
    if m.distance < 0.75*n.distance:
        good_matches.append(m)

good_matches = sorted(good_matches, key = lambda x:x.distance)
img_good_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches[:200], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.namedWindow('MatchedPoints200', cv2.WINDOW_NORMAL)
cv2.imshow('MatchedPoints200', img_good_matches)
cv2.waitKey(0)

-1

### 最近距離比から選出した良いマッチングペアにより基礎行列Fを計算する。算出したFにより外れポイントを外す

In [30]:
pts1 = []
pts2 = []

for i,m in enumerate(good_matches):
    pts1.append(kp1[m.queryIdx].pt)
    pts2.append(kp2[m.trainIdx].pt)

import numpy as np
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.RANSAC)

pts1_inliers = pts1[mask.ravel() == 1]
pts2_inliers = pts2[mask.ravel() == 1]

good_matches_inliers = [m for i,m in enumerate(good_matches[:200]) if mask[i,0] == 1]

img_good_matches_inliers = cv2.drawMatches(img1,kp1,img2,kp2,good_matches_inliers[:200],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.namedWindow('GoodMatchedPoints', cv2.WINDOW_NORMAL)
cv2.imshow('GoodMatchedPoints', img_good_matches_inliers)
cv2.waitKey(0)


-1

### エピポーラ線を描画する。
- エピポーラ線 ax+by+c=0の(a,b,c)は(r[0],r[1],[r[2]])
- エピポーラ線の両端はx=0の時とx=画像幅の時

In [31]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c,ch = img1.shape
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

pts1_inliers_200 = pts1_inliers[0:200,:]
pts2_inliers_200 = pts2_inliers[0:200,:]

lines1 = cv2.computeCorrespondEpilines(pts2_inliers_200.reshape(-1,1,2),2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1.copy(),img2.copy(),lines1,pts1,pts2)
lines2 = cv2.computeCorrespondEpilines(pts1_inliers_200.reshape(-1,1,2),1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2.copy(),img1.copy(),lines2,pts2,pts1)
img35 = np.hstack((img3,img5))
cv2.namedWindow('EpipoleLines', cv2.WINDOW_NORMAL)
cv2.imshow('EpipoleLines', img35)
cv2.waitKey(0)

-1

### さらに、Fで外れ値を削除した後の200個のペアを選んでにより基礎行列Fを計算して、エピポーラ線を可視化する

In [32]:
F, mask = cv2.findFundamentalMat(pts1[0:200,:], pts2[0:200,:], cv2.FM_RANSAC)

pts1 = pts1[0:200,:]
pts2 = pts2[0:200,:]
pts1_inliers = pts1[mask.ravel() == 1]
pts2_inliers = pts2[mask.ravel() == 1]

pts1_inliers_200 = pts1_inliers[0:200,:]
pts2_inliers_200 = pts2_inliers[0:200,:]

lines1 = cv2.computeCorrespondEpilines(pts2_inliers_200.reshape(-1,1,2),2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1.copy(),img2.copy(),lines1,pts1,pts2)
lines2 = cv2.computeCorrespondEpilines(pts1_inliers_200.reshape(-1,1,2),1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2.copy(),img1.copy(),lines2,pts2,pts1)
img35 = np.hstack((img3,img5))
cv2.namedWindow('GoodEpipoleLines', cv2.WINDOW_NORMAL)
cv2.imshow('GoodEpipoleLines', img35)
cv2.waitKey(0)

-1